In [ ]:
!gdown 1Rt0I7Svrx77tFMCsNubEQ-cDY8hD-iCk

In [ ]:
!unzip r_peaks.zip

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from tqdm import tqdm

In [4]:
labels = pd.read_csv('r_peaks/labels.csv')

In [30]:
def euclidean_distance(matrix1, matrix2):
    diff = matrix1 - matrix2
    squared_diff = diff ** 2
    sum_squared_diff = np.sum(squared_diff)
    euclidean_dist = np.sqrt(sum_squared_diff)
    return euclidean_dist

def compare_matrix(name, matrixes):
    target = matrixes[name]
    buffer = {}
    min_dist = float('inf')

    for mat in matrixes:
        if mat[:8] == target[:8]:
            continue

        dist = euclidean_distance(matrixes[mat], target)

        if 0 < dist < min_dist:
            buffer = {'name': mat, 'hr': matrixes[mat]}
            min_dist = dist

    return buffer

In [35]:
counts = labels.groupby('result_class').count().reset_index()

In [ ]:
be_in_same = []
count = 0

for clas in range(list(counts[counts.record_name < 25000].result_class)):
    print('Class: ', clas)
    names = labels[labels.result_class == clas].record_name
    matrixes = {}

    for name in tqdm(names):
        hr = np.load(f'r_peaks/signals/{name}.npy')
        matrixes[name] = hr

    for name in tqdm(names):
        same = compare_matrix(name, matrixes)

        if (same['name'], name) in be_in_same:
            continue

        be_in_same.append((name, same['name']))

        hr1 = np.load(f'r_peaks/signals/{name}.npy')
        hr2 = same['data']
        new = (hr1 + hr2) / 2

        new_name = f"{name.replace('_hr_', '_')}_aug_{same['name'].replace('_hr_', '_')}"

        np.save(f'r_peaks/signals/{new_name}.npy', new)
        labels = pd.concat([labels, pd.DataFrame(data={'record_name': [new_name], 'myocard_type': [clas]})])

    clear_output(wait=True)